In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df.index = pd.to_datetime(df.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]
display(df.head(1), df.tail(1))
df.shape


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2010-06-01 00:00:00+00:00,-0.195535,-0.431203,-0.701336,-0.140981,-0.470019,-0.252915,-0.504554,-0.271715,0.354467,0.168679,...,-0.372762,0.185407,-0.157289,-0.047979,-0.305857,-0.55999,-0.335956,0.293033,0.037194,-0.217444


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,-0.05858,-0.048583,0.019215,0.010852,-0.092832,0.009434,0.079595,0.068199,0.036857,-0.023727,...,0.065983,0.070777,-0.053466,-0.080965,-0.077221,-0.004721,-0.121889,0.041164,0.052337,0.119222


(74593, 28)

In [3]:
SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]



In [4]:
def mean_prediction(df, rate=24):
    data = pd.DataFrame([])
    for sym in SYMBOLS:
        data[f'{sym}_tend'] = df[sym].rolling(rate).mean()
    data.index = df.index
    return data
        
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    return data

In [5]:
rand = random_prediction(df)
mu = mean_prediction(df)

In [6]:
def condition(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0

for sym in mu.columns:
    mu[sym] = mu[sym].map( lambda x: condition(x))
    
# mu.reset_index(inplace=True, drop=True)

display(rand.shape, mu.shape)


(74593, 28)

(74593, 28)

In [7]:
pred_last = rand.iloc[-120:]
pred = rand.iloc[-1:]
pred_last
pred = abs(pred)



In [8]:
def pairs_correlation(df, rate=120):
    
    data = pd.DataFrame([])
    for pair in range(len(df.columns)):
        for second_pair in range(pair, len(df.columns)):
            data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
    data.index = df.index
    data = data.iloc[-1:]
    return data


def weighted_volatility(volatility):
    weight = pd.DataFrame([])
    volatility = volatility.copy()
    volatility_T = volatility.T
    weight = volatility_T/ volatility_T.mean()
    return  weight

In [9]:
w = weighted_volatility(pred)
wt = w.T

In [10]:
cor = pairs_correlation(pred_last)


/tmp/ipykernel_1516/3071229965.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
/tmp/ipykernel_1516/3071229965.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
/tmp/ipykernel_1516/3071229965.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [679]:
cor.shape

(1, 406)

In [11]:
cor = cor.iloc[-1:,]
cort = cor.T
cort.columns = ["dot"]
cort = cort[round(cort["dot"], 6) != 1.000000]
w.columns = ["dot"]
wt = w.T
display(cort.iloc[78:102], wt)


,dot
3_4_corr,0.023580
3_5_corr,0.085708
3_6_corr,-0.167437
3_7_corr,-0.032505
3_8_corr,0.015381
3_9_corr,0.112485
3_10_corr,-0.004419
3_11_corr,0.069005
3_12_corr,-0.052928
3_13_corr,0.101978


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
dot,0.979277,0.693818,0.866351,1.803933,1.192528,0.868928,1.089745,0.924965,0.845829,1.443402,...,1.438982,0.092826,1.554888,0.876313,0.199778,1.164805,1.71857,0.718804,0.408722,0.596769


In [12]:
def risk_volatility(pred_vol, correlations):
    
    cor_vol_mean = []
    
    start = 0
    end = len(pred_vol.columns)
    for pair in range(len(pred_vol.columns)):
        end = end - 1 
        vol = np.array(pred_vol.iloc[:,pair])
        
        data1 = pred_vol.iloc[:,pair:]
        data2 = correlations.iloc[:,start:end]
        display(data2)
        start = end+start
        
                
        res = data1.T.dot(data2)
        
        display(res.mean() + vol)
        # cor_vol_mean.append(vol.mean)
    return 


In [661]:
display(cort.T, wt)

,0_1_corr,0_2_corr,0_3_corr,0_4_corr,0_5_corr,0_6_corr,0_7_corr,0_8_corr,0_9_corr,0_10_corr,...,23_24_corr,23_25_corr,23_26_corr,23_27_corr,24_25_corr,24_26_corr,24_27_corr,25_26_corr,25_27_corr,26_27_corr
correlation,-0.174083,0.052609,-0.178924,0.08179,-0.097132,0.09572,-0.058109,0.144871,0.063495,0.053768,...,0.089723,-0.027544,-0.040532,0.086227,0.042118,-0.084304,-0.259751,-0.094132,0.100786,-0.018836


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.075729,0.611049,1.371241,0.401231,1.638511,0.098102,1.007713,0.500091,0.152478,1.382614,...,0.681344,0.827289,1.82473,1.536075,0.038547,0.161907,1.646442,0.843942,0.323906,1.093536


In [ ]:
def risk_volatility2(pred_vol, correlations):
    cor_vol = []
    cor_vol_mean = []
    number = len(pred_vol.columns)
    for pair in range(len(pred_vol.columns)):
        for cor in range(len(correlations.columns)):
        
            vol = np.array(pred_vol[pair])
            print(vol)
            
            data1 = pred_vol[pair+1]
            print(data1.shape)
            data2 = correlations.iloc[:,pair*number:number*(pair+1)-1].T
            # print(data2)
            print(data2.shape)
            # print(data2)
            res = data1*data2
            
            display(res.mean() + vol)
            # cor_vol_mean.append(vol.mean)
    return cor_vol_mean


In [ ]:
m = risk_volatility2(wt, cort.T)
m


In [31]:
def correlation_pairs(df, rate=120):
    
    data = pd.DataFrame([])
    for pair in df.columns:
        for second_pair in df.columns:
            data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(120).corr(df[second_pair])
    data.index = df.index
    data = data.iloc[-1:].T
    data.columns = ["correlation"]
    data = data[round(data["correlation"],6) != 1.000000]
    return data



In [33]:
a = correlation_pairs(pred_last)

/tmp/ipykernel_1516/4232714446.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(120).corr(df[second_pair])
/tmp/ipykernel_1516/4232714446.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(120).corr(df[second_pair])
/tmp/ipykernel_1516/4232714446.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all c

In [39]:
display(a.T.iloc[:,52:60], wt)

,AUD_CHF_USD_CHF_corr,AUD_CHF_USD_JPY_corr,AUD_JPY_AUD_CAD_corr,AUD_JPY_AUD_CHF_corr,AUD_JPY_AUD_NZD_corr,AUD_JPY_AUD_USD_corr,AUD_JPY_CAD_CHF_corr,AUD_JPY_CAD_JPY_corr
correlation,0.063981,-0.152869,0.174553,-0.093876,-0.066694,-0.214167,-0.10546,0.009738


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
dot,0.979277,0.693818,0.866351,1.803933,1.192528,0.868928,1.089745,0.924965,0.845829,1.443402,...,1.438982,0.092826,1.554888,0.876313,0.199778,1.164805,1.71857,0.718804,0.408722,0.596769


In [ ]:
def correlation_volatility(pred_vol, correlations):
    
     for pair in range(len(pred_vol.columns)):
        
        vol = np.array(pred_vol.iloc[:,pair])
        
        data1 = pred_vol.iloc[:,pair:]
        data2 = correlations.iloc[:,start:end]
        display(data2)
        start = end+start
        
                
        res = data1.T.dot(data2)
        
        display(res.mean() + vol)
        # cor_vol_mean.append(vol.mean)
    return 
